In [8]:
# [1] Install Required Packages
!pip install datasets
!pip install scikit-learn
!pip install ibm-watson-machine-learning==1.0.312

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 92.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 14.4 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.6-py3-none-any.whl size=77230 sha256=0f6621953459b78fd35bb90a903219d920c6e86c7b3c788595a2a8571efcdfeb
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/17/8c/76/f9472a53a2a80da414dd53ca55f08dbfe48ed6de76e51e3d5f
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.13.6-py3-none-any.whl size=661464 sha256=105de9e88bc

In [15]:
# [2] Import libraries
import os, types, getpass
import pandas as pd
from botocore.client import Config
import ibm_boto3
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model

In [16]:
# [3] Set IBM Watsonx Credentials
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",  # change if not us-south
    "apikey": getpass.getpass("Enter your IBM Cloud API key: ")
}

Enter your IBM Cloud API key:  ········


In [17]:


# [4] Set the correct project ID (NO &context!)
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id only (no &context): ")

In [18]:
# [5] Connect to IBM COS and read CSV files
bucket = 'carrentalfeedback'  # your bucket name

# Define function to read CSV from COS
def read_cos_csv(object_key):
    cos_client = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=credentials["apikey"],
        ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
        config=Config(signature_version='oauth'),
        endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud'
    )

    body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
    if not hasattr(body, "_iter"): 
        body.iter_ = types.MethodType(lambda self: 0, body)
    return pd.read_csv(body)

# [6] Load train and test data
train_data = read_cos_csv('car_rental_train.csv')
test_data = read_cos_csv('car_rental_test.csv')

print("Train:", train_data.shape)
print("Test:", test_data.shape)

Train: (5, 2)
Test: (5, 2)


In [19]:
# [7] Define the foundation model to use
model_id = ModelTypes.FLAN_UL2

parameters = {
    GenParams.MAX_NEW_TOKENS: 15
}

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

# [8] Predict Satisfaction using Watsonx
satisfaction_instruction = """
Was customer satisfied?\n
comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.\nsatisfaction: 0\n\n
"""

results = []
comments = list(test_data['Customer Feedback'])  # match your column

for input_text in comments:
    results.append(model.generate_text(prompt=satisfaction_instruction + input_text))

In [20]:
# [9] Show predicted satisfaction
for comment, result in zip(comments, results):
    print("\nCOMMENT:", comment)
    print("PREDICTION:", result)

# [10] Predict Business Area
business_area_instruction = """
Find the business area of the customer e-mail.
Choose business area from:
'Product: Functioning', 'Product: Pricing and Billing', 'Service: Accessibility',
'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts'.\n
comment: I do not understand why I have to pay additional fee if vehicle is returned without a full tank.\n
business area: 'Product: Pricing and Billing'\n\n
"""

results_area = []
for input_text in comments:
    results_area.append(model.generate_text(prompt=business_area_instruction + input_text))

# [11] Show predicted business area
for comment, result in zip(comments, results_area):
    print("\nCOMMENT:", comment)
    print("BUSINESS AREA:", result)



COMMENT: The booking experience was fast and the staff was friendly.
PREDICTION: satisfaction: 4

COMMENT: I faced issues while starting the car, very disappointing.
PREDICTION: dissatisfied

COMMENT: The rental price was affordable and no hidden charges.
PREDICTION: satisfaction: 3

COMMENT: Long queue at the pickup counter, wasted a lot of time.
PREDICTION: dissatisfied

COMMENT: Smooth driving experience, clean car and quick return.
PREDICTION: satisfaction: 5

COMMENT: The booking experience was fast and the staff was friendly.
BUSINESS AREA: Service: Attitude

COMMENT: I faced issues while starting the car, very disappointing.
BUSINESS AREA: Product: Functioning

COMMENT: The rental price was affordable and no hidden charges.
BUSINESS AREA: Product: Pricing and Billing

COMMENT: Long queue at the pickup counter, wasted a lot of time.
BUSINESS AREA: Service: Attitude

COMMENT: Smooth driving experience, clean car and quick return.
BUSINESS AREA: Product: Functioning
